In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
### load scores calculated through cross validation
with open('baseline_reports_d.p','rb') as f:
    scores=pickle.load(f)

In [3]:
#[sum(x,y) for x, y in zip(scores[0][2][0],scores[1][2][0])]
score=[]

study=[]
for i in range(0,10):
    study.append(scores[i][1])
    score.append(scores[i][2][0]+[scores[i][2][1]]+[scores[i][2][2]]+[scores[i][2][3]])
print(np.mean(np.array(score),0))
print(study)

[0.27233238 0.17122531 0.12285236 0.09288897 0.13432849 0.29311013
 0.26957508]
['s52334805', 's50691028', 's52334805', 's52334805', 's52334805', 's52334805', 's52334805', 's52334805', 's52334805', 's52334805']


In [4]:

###### chexpert labeler based accuracy

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,f1_score, recall_score, precision_score, average_precision_score



orig_data=pd.read_csv('../data/mimic-cxr-2.0.0-chexpert.csv')

#print([i for i,x in enumerate(orig_data['study_id']=='51523013') if(x)])
#np.where(orig_data['study_id']==51523013)[0]

data=orig_data.iloc[:,2:]

data=data.fillna(0)

data = data.replace(-1, 0)



kf = KFold(n_splits=10)
kf.get_n_splits(data)

ten_fold_acc=[]

i=0
for train_index,test_index in kf.split(data):
    idx=study[i].lstrip('s')
    print(idx)
    baseline_idx=np.where(orig_data['study_id']==int(idx))[0]
    
    i+=1
    
    baseline = data.iloc[baseline_idx,:]
    
    X_train, X_test = data.iloc[train_index,:], data.iloc[test_index,:]
    X_train=X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    
    
    baseline=baseline *1.0
    #print(baseline)
    pos=X_test.multiply(np.array(baseline),axis='columns')
    #print(pos.head(3))
    neg=(1 - X_test).multiply(np.array(1-baseline),axis='columns')
    comp=pos + neg
    
    #print(comp)
    
    accu = np.mean(comp,axis=1)
    
    ten_fold_acc.append(np.mean(accu))
    

print(np.mean(ten_fold_acc))
    
    
    
        

52334805
50691028
52334805
52334805
52334805
52334805
52334805
52334805
52334805
52334805
0.8432450895713132
